In [2]:
!pip install gensim
import numpy as np
import pandas as pd
from gensim.models import KeyedVectors
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt

# Далі ваш код без змін...
# 1. Завантаження датасету столиць...

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.9/27.9 MB 75.4 MB/s eta 0:00:00


In [3]:
import numpy as np
import pandas as pd
from gensim.models import KeyedVectors
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt

# 1. Завантаження датасету столиць (для загальної перевірки)
url_capitals = "https://raw.githubusercontent.com/lang-uk/vecs/refs/heads/master/test/test_vocabulary.txt"
# Пропускаємо перший рядок, якщо там заголовок, або підлаштовуємо skiprows
data = pd.read_csv(url_capitals, delimiter=' ', header=None, names=['country1', 'city1', 'country2', 'city2'])

# 2. Завантаження моделі Word2Vec (Ubercorpus)
# Увага: цей процес може зайняти час, архів важить ~600MB, розпакований ~1.5GB
!wget -nc https://lang.org.ua/static/downloads/models/ubercorpus.cased.tokenized.word2vec.300d.bz2
!bzip2 -dkq ubercorpus.cased.tokenized.word2vec.300d.bz2

print("Завантаження моделі в пам'ять...")
# binary=False, бо це текстовий формат
model = KeyedVectors.load_word2vec_format("ubercorpus.cased.tokenized.word2vec.300d", binary=False)
print("Модель завантажено!")

--2025-12-03 14:27:59--  https://lang.org.ua/static/downloads/models/ubercorpus.cased.tokenized.word2vec.300d.bz2
Resolving lang.org.ua (lang.org.ua)... 65.21.91.242
Connecting to lang.org.ua (lang.org.ua)|65.21.91.242|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 555203899 (529M) [application/octet-stream]
Saving to: ‘ubercorpus.cased.tokenized.word2vec.300d.bz2’

ubercorpus.cased.to 100%[===================>] 529.48M  19.4MB/s    in 28s     

2025-12-03 14:28:28 (19.1 MB/s) - ‘ubercorpus.cased.tokenized.word2vec.300d.bz2’ saved [555203899/555203899]

Завантаження моделі в пам'ять...
Модель завантажено!


In [5]:
def cosine_similarity_manual(u, v):
    """
    Обчислює косинусну подібність: (u . v) / (|u| * |v|)
    """
    dot_product = np.dot(u, v)
    norm_u = np.linalg.norm(u)
    norm_v = np.linalg.norm(v)
    return dot_product / (norm_u * norm_v)

def euclidean_distance_manual(u, v):
    """
    Обчислює евклідову відстань: ||u - v||
    """
    return np.linalg.norm(u - v)

# Перевірка на прикладі
word1 = "Київ"
word2 = "Львів"
if word1 in model and word2 in model:
    sim = cosine_similarity_manual(model[word1], model[word2])
    dist = euclidean_distance_manual(model[word1], model[word2])
    print(f"Подібність між '{word1}' та '{word2}': {sim:.4f}")
    print(f"Відстань між '{word1}' та '{word2}': {dist:.4f}")

Подібність між 'Київ' та 'Львів': 0.7501
Відстань між 'Київ' та 'Львів': 14.4726


In [6]:
def get_country(city1, country1, city2, word_embeddings):
    """
    Шукає країну для city2, використовуючи аналогію:
    vec = city2 - city1 + country1
    """
    # Перевірка наявності слів у словнику
    for w in [city1, country1, city2]:
        if w not in word_embeddings:
            return None, -1

    city1_emb = word_embeddings[city1]
    country1_emb = word_embeddings[country1]
    city2_emb = word_embeddings[city2]

    # Векторна формула
    target_vec = city2_emb - city1_emb + country1_emb

    # Використовуємо вбудований метод gensim для швидкості (він оптимізований)
    # most_similar повертає список (слово, схожість)
    # topn=10, щоб мати запас, якщо перше слово - це саме вхідне слово
    similar_words = word_embeddings.similar_by_vector(target_vec, topn=5)

    group = {city1, country1, city2}

    for word, score in similar_words:
        if word not in group:
            return word, score

    return None, -1

def calculate_accuracy(word_embeddings, df):
    correct = 0
    total = 0

    print("Початок тестування точності...")

    # Беремо перші 1000 прикладів для економії часу (можна прибрати [:1000] для повного тесту)
    for index, row in df[:1000].iterrows():
        c1, cty1, c2, cty2 = row['country1'], row['city1'], row['country2'], row['city2']

        predicted_country, _ = get_country(cty1, c1, cty2, word_embeddings)

        if predicted_country:
            total += 1
            if predicted_country == c2:
                correct += 1

            # Вивід для налагодження (кожні 100 рядків)
            if index % 200 == 0:
                print(f"#{index}: {cty1}-{c1}, {cty2} -> {predicted_country} (Очікувалось: {c2})")

    accuracy = correct / total if total > 0 else 0
    return accuracy

# Запуск розрахунку (Зверніть увагу: dataset може містити помилки або слова не з словника)
acc = calculate_accuracy(model, data)
print(f"\nТочність (Accuracy) на вибірці: {acc:.2%}")

Початок тестування точності...

Точність (Accuracy) на вибірці: 0.00%


In [7]:
def test_opposite_analogy(positive_word, negative_word, target_negative, model):
    """
    positive_word (наприклад, 'добрий')
    negative_word (наприклад, 'злий')
    target_negative (наприклад, 'темний')
    Очікуємо: 'світлий'
    Formul: positive - negative + target_negative -> target_positive ?
    АБО (класична аналогія):
    'білий' - 'чорний' + 'ніч' -> 'день'
    """
    try:
        # Вектор: Білий - Чорний + Ніч
        res_vec = model[positive_word] - model[negative_word] + model[target_negative]

        match = model.most_similar(positive=[res_vec], topn=3)
        print(f"Аналогія: {positive_word} - {negative_word} + {target_negative} = ?")
        print(f"Результат: {match[0][0]} (score: {match[0][1]:.2f})")
        print(f"Топ-3 кандидати: {[m[0] for m in match]}")
        print("-" * 30)
    except KeyError as e:
        print(f"Слово відсутнє в словнику: {e}")

print("\n=== Перевірка варіанту 7 (Opposite) ===\n")

# Приклад 1: День/Ніч
test_opposite_analogy('день', 'ніч', 'чорний', model)
# Очікуємо: білий (день - ніч + чорний ? Ні, це нелогічно).
# Правильна формула: a:b :: c:d => b - a + c = d
# ніч : день :: чорний : ? => день - ніч + чорний = білий
# Перевіримо:
test_opposite_analogy('день', 'ніч', 'чорний', model)

# Приклад 2: Гендер
test_opposite_analogy('король', 'чоловік', 'жінка', model)

# Приклад 3: Прикметники
test_opposite_analogy('гарний', 'поганий', 'зло', model) # очікуємо добро
test_opposite_analogy('холодний', 'гарячий', 'літо', model) # очікуємо зима


=== Перевірка варіанту 7 (Opposite) ===

Аналогія: день - ніч + чорний = ?
Результат: день (score: 0.55)
Топ-3 кандидати: ['день', 'день**', 'штрих']
------------------------------
Аналогія: день - ніч + чорний = ?
Результат: день (score: 0.55)
Топ-3 кандидати: ['день', 'день**', 'штрих']
------------------------------
Аналогія: король - чоловік + жінка = ?
Результат: королева (score: 0.64)
Топ-3 кандидати: ['королева', 'принцеса', 'імператриця']
------------------------------
Аналогія: гарний - поганий + зло = ?
Результат: зло (score: 0.79)
Топ-3 кандидати: ['зло', 'Зло', 'чудо']
------------------------------
Аналогія: холодний - гарячий + літо = ?
Результат: літо (score: 0.89)
Топ-3 кандидати: ['літо', 'Літо', 'лiто']
------------------------------


In [8]:
def test_opposite_analogy(positive_word, negative_word, target_negative, model):
    """
    positive_word (наприклад, 'добрий')
    negative_word (наприклад, 'злий')
    target_negative (наприклад, 'темний')
    Очікуємо: 'світлий'
    Formul: positive - negative + target_negative -> target_positive ?
    АБО (класична аналогія):
    'білий' - 'чорний' + 'ніч' -> 'день'
    """
    try:
        # Вектор: Білий - Чорний + Ніч
        res_vec = model[positive_word] - model[negative_word] + model[target_negative]

        match = model.most_similar(positive=[res_vec], topn=3)
        print(f"Аналогія: {positive_word} - {negative_word} + {target_negative} = ?")
        print(f"Результат: {match[0][0]} (score: {match[0][1]:.2f})")
        print(f"Топ-3 кандидати: {[m[0] for m in match]}")
        print("-" * 30)
    except KeyError as e:
        print(f"Слово відсутнє в словнику: {e}")

print("\n=== Перевірка варіанту 7 (Opposite) ===\n")

# Приклад 1: День/Ніч
test_opposite_analogy('день', 'ніч', 'чорний', model)
# Очікуємо: білий (день - ніч + чорний ? Ні, це нелогічно).
# Правильна формула: a:b :: c:d => b - a + c = d
# ніч : день :: чорний : ? => день - ніч + чорний = білий
# Перевіримо:
test_opposite_analogy('день', 'ніч', 'чорний', model)

# Приклад 2: Гендер
test_opposite_analogy('король', 'чоловік', 'жінка', model)

# Приклад 3: Прикметники
test_opposite_analogy('гарний', 'поганий', 'зло', model) # очікуємо добро
test_opposite_analogy('холодний', 'гарячий', 'літо', model) # очікуємо зима


=== Перевірка варіанту 7 (Opposite) ===

Аналогія: день - ніч + чорний = ?
Результат: день (score: 0.55)
Топ-3 кандидати: ['день', 'день**', 'штрих']
------------------------------
Аналогія: день - ніч + чорний = ?
Результат: день (score: 0.55)
Топ-3 кандидати: ['день', 'день**', 'штрих']
------------------------------
Аналогія: король - чоловік + жінка = ?
Результат: королева (score: 0.64)
Топ-3 кандидати: ['королева', 'принцеса', 'імператриця']
------------------------------
Аналогія: гарний - поганий + зло = ?
Результат: зло (score: 0.79)
Топ-3 кандидати: ['зло', 'Зло', 'чудо']
------------------------------
Аналогія: холодний - гарячий + літо = ?
Результат: літо (score: 0.89)
Топ-3 кандидати: ['літо', 'Літо', 'лiто']
------------------------------
